In [1]:
import json
import logging
import os
import pandas as pd
import random
import re
import sys
from datetime import date
from dotenv import load_dotenv
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.utils import GPT

In [2]:
# params
# st_mdl = "gpt-3.5-turbo-1106" # model string
st_mdl = "gpt-4-1106-preview"
st_task = "race" # name of task

# logging
## today's date
st_today = date.today().strftime("%Y_%m_%d")

## file name:
st_fn = f"{st_today}_{st_task}_n200_{st_mdl}"

## Configure the logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Create a file handler and set the formatter
file_handler = logging.FileHandler(f"results/{st_fn}.log")
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

## Create a logger instance and add the file handler
logger = logging.getLogger(__name__)
logger.addHandler(file_handler)

In [3]:
# load api key from .env and create GPT object
load_dotenv()  # Load environment variables from .env file
openai_api_key = os.getenv("OPENAI_API_KEY")
gpt = GPT(openai_api_key, logger=logger, model=st_mdl)

# Load into a dictionary
data = []
dir_data = 'data/high'
for fn in os.listdir(dir_data):
    if fn.endswith('.txt'):
        fp = os.path.join(dir_data, fn)
        with open(fp, 'r') as f:
            task = [json.loads(l) for l in f]
            assert len(task) == 1
            data.append(task[0])

print(len(data))


# dataset-specific adjustments (we eventually want a list of dicts of format [{"id": "xx", "question": "xxx", "context": "xxx"}, ...]

# pre-process data

data = [{'questions': d['questions'], 'context': d['article'], 'options': d['options'], 'answers': d['answers']} for d in data]
    
# data = pd.read_csv('data/24game.csv').to_dict(orient='records') # load CSV into list of dicts

1045


In [4]:
def compare_strings(str1, str2):
    """
    This function compares two strings after converting them to lowercase and stripping leading and trailing spaces.
    """
    return str1.lower().strip() == str(str2).lower().strip()

def check_answer(pred, gold):
    """
    This function compares first int in str pred with str gold
    """
    try:
        return next((c for c in pred if c.isupper()), None) == gold
        # return idx_to_char(int(re.search(r'\d+', pred).group())) == gold
    except:
        return False

def idx_to_char(idx):
    """
        Map 0 to A, 1 to B, 2 to C, etc.
    """
    return chr(idx + ord('A'))

def check_bool(str1, bool2):
    """
        Checks if string str1 equal to bool bool2
    """
    try:
        return bool(str1) == bool2
    except:
        return False
        

In [5]:
# visualize dataset
# for r in data:
#     if len(r['questions']) >= 2:
#         print(r['questions'], r['answers'], r['options'])

df = pd.json_normalize(data)
df.head()

,questions,context,options,answers
0,"[Mr. Torres became blind when _ ., Mr. Torre...","You could feel sorry for Alberto Torres, who i...","[[his daughter was just born, he was thirteen ...","[A, C, D, B, C, D]"
1,[From Para.1 we learn that lying is very _ ...,Studies show that you may be lied to every day...,"[[harmful, easy, interesting, common], [the li...","[D, C, B, A, D]"
2,"[What will be tested in the new SAT?, The new ...",The SAT is one of two major tests for the entr...,"[[All the different meanings of a word., The r...","[D, D, B]"
3,[We can learn from the beginning of the passag...,It was 1961 and I was in the fifth grade. My m...,[[the author and his brother had done well in ...,"[C, A, B, D]"
4,[What does the father mainly want to say to hi...,"Dear son,\nThe day that you see me old and I a...","[["" Help me to understand my life."", "" Teach y...",[D]


## Generator

In [7]:
predictions = [] # list of predictions, reset after each set of runs
indices = [] # question numbers
answers = [] # list of gold answers
evals = [] # whether prediction matches answer
questions = [] # list of questions


# Evaluate
num_questions = 100 # number of questions to run on
indices = random.sample(range(len(data)), num_questions)
lst_indices = []
for idx in indices:
    item = data[idx]
    # if item['level'] == 'hard':
    # gold, question, context = item['answer'], item['question'], item['context']
    lst_gold, lst_q, lst_opts, context = item['answers'], item['questions'], item['options'], item['context']
    for gold, q, opts in zip(lst_gold, lst_q, lst_opts):
        # concat question with options
        question = f"Context: {context}\n Question: {q}\n"
        choices = []
        for i in range(len(opts)):
            choices.append(idx_to_char(i))
            question += f"{choices[-1]}.{opts[i]}\n"
        logger.info(question)
        
        msg = [{"role": "system", "content": f"Please answer the following multiple choice question given some context. Respond only with one of {choices} corresponding to the correct answer```"}, {"role": "user", "content": question}]
        pred = gpt.gen(msgs=msg).content
        predictions.append(pred)
        answers.append(gold)
        # evals.append(compare_strings(predictions[-1], gold))
        evals.append(check_answer(predictions[-1], gold))
        questions.append(question)
        lst_indices.append(idx)
        print(predictions[-1])

    

# Print the predictions and answers
logger.info(predictions)
# logger.info(answers)
percent_correct = (sum(evals) / len(evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# create a dataframe with the data
df = pd.DataFrame({'Index': lst_indices, 'Question': questions, 'Prediction': predictions, 'Answer': answers, 'Eval': evals})
# df = pd.DataFrame({'Index': indices, 'Question': questions, 'Prediction': predictions, 'Eval': evals})

# save the datafrme to csv
df.to_csv(f'results/{st_fn}.csv', index=False)

2024-02-13 22:19:24,376 - INFO - Context: Some nations think they must have more and more babies, more and more people, if they are to remain strong and free.
Actually, this is not so. Very often in history, small nations have conquered large ones. It's not so much the size of the army as its organization and the technical level of its weapons. Thus, Greece took over Persia in the 300s B.C., Great Britain took over India in the 1700s, even though Persia and India had far bigger population than Greece and Great Britain.
If a nation wishes to avoid being dominated by its neighbors, its best chance is to raise its standard of living and its level of technology. _ can be done best by not allowing its population to grow to such a point that it is sunk in misery and poverty. In fact, the worst way in which a nation can try to avoid being dominated by its neighbor is to increase its population to the point of misery and poverty.
If every nation tries to compete with its neighbors by raising i

A


2024-02-13 22:19:26,717 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 22:19:26,719 - INFO - Context: Some nations think they must have more and more babies, more and more people, if they are to remain strong and free.
Actually, this is not so. Very often in history, small nations have conquered large ones. It's not so much the size of the army as its organization and the technical level of its weapons. Thus, Greece took over Persia in the 300s B.C., Great Britain took over India in the 1700s, even though Persia and India had far bigger population than Greece and Great Britain.
If a nation wishes to avoid being dominated by its neighbors, its best chance is to raise its standard of living and its level of technology. _ can be done best by not allowing its population to grow to such a point that it is sunk in misery and poverty. In fact, the worst way in which a nation can try to avoid being dominated by its neighbor is to increase it

B


2024-02-13 22:19:27,466 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 22:19:27,468 - INFO - Context: Joe Temper should have known better. After all, he works for a large auto insurance company. It won't hurt to leave the key in the truck this once, he thought, as he filled his gas tank at a self-service gas station. But minutes later, as he was paying the money, he saw the truck being driven away.
In 2010, 1.6 million motor vehicles were stolen in the United States --one every 20 seconds. If present trends continue, experts predict vehicle thefts   could be over two million every year by the end of the decade.
Vehicle theft is a common phenomenon, which has a direct effect on over four million people a year. The cost is astonishing.
Many police officials blame professional thieves for the large number of the thefts. It is a major moneymaker for organized crime. Typically, stolen cars are taken to pieces and the parts sold to individ

...

D


2024-02-13 22:23:30,992 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 22:23:30,994 - INFO - Context: With thousands of waterfalls across the world, it is difficult to choose the best but the following list contains some of the world's most amazing and beautiful ones!
Iguazu Falls-The Iguazu Falls is located on the border of Brazil and Argentina and it is well-known for its scenic beauty. Though it is just about 89 meters high, it ranks as the fourth best waterfall in the world because of its beauty. The U-shaped cliff in the waterfall which is known as the Devil's throat is the most striking feature of the waterfall.
Angel Falls-The Angel Falls, named after its discoverer James Crawford Angel, is the world's highest waterfalls, falling from a height of 979 meters. It is located in the Bolivar state in Venezuela. A unique feature of the falls is that most of the water falling down gets blown away by the winds as mist . The fallen wat

C


2024-02-13 22:23:31,775 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-13 22:23:31,777 - INFO - ['A', 'B', 'A', 'C', 'B', 'D', 'A', 'C', 'A', 'B', 'A', 'B', 'C', 'D', 'B', 'B.Optimistic .', 'B', 'D', 'C', 'A', 'D', 'C', 'A', 'D', 'A', 'B', 'D', 'C', 'D', 'A', 'A', 'A', 'B', 'B.72 hours', 'A', 'B', 'C', 'B', 'A', 'C', 'D', 'C', 'A', 'C', 'B', 'B', 'C', 'C', 'B', 'D', 'D', 'A', 'B', 'C', 'B', 'B.$110.', 'C', 'B.', 'B', 'C', 'D', 'B', 'C', 'B', 'D', 'A', 'B', 'C', 'D', 'C', 'B', 'D', 'A', 'B', 'D', 'C', 'D', 'B.Enthusiastic.', 'C', 'B. Environment.', 'C', 'A.150', 'D', 'C.Disaster Warning System', 'C', 'A. Sanyuan', "C. Deng Nannan began to drink milk because of her mother's advice.", 'B', 'D', 'B', 'C', 'A. Cinnamon Boutique', 'D', 'B', 'C', 'B', 'B', 'D', 'B', 'D', 'C', 'D', 'C', 'B', 'B', 'C', 'B', 'C', 'A', 'A', 'D', 'C.', 'B', 'C. study the society that created Hitler', 'B', 'A', 'D', 'D', 'C', 'B.', 'C', 'C', 'D', 'B', 'D', 'A', 'D. 

A
Percentage of True values: 90.21406727828746%


In [23]:
# lst_indices = []
# for idx in indices:
#     lst_indices.extend([idx]*len(data[idx]['questions']))
    
# df = pd.DataFrame({'Index': lst_indices, 'Question': questions, 'Prediction': predictions, 'Answer': answers, 'Eval': evals})

# # save the datafrme to csv
# df.to_csv(f'results/{st_fn}.csv', index=False)

In [29]:
# df = pd.read_csv(f"results/{st_fn}.csv")
# # df['Answer'] = df['Answer'].apply(lambda x: x.split('\n#### ')[-1])
# df['Eval'] = df.apply(lambda row: check_answer(row['Prediction'], row['Answer']), axis=1)

# df.to_csv(f'results/{st_fn}.csv', index=False)
# df.head()

,Index,Question,Prediction,Answer,Eval
0,119,Context: The worst days of any summer are the ...,1.hated rainy days,B,False
1,119,Context: The worst days of any summer are the ...,2. had no brothers or sisters,C,False
2,119,Context: The worst days of any summer are the ...,3. he can better deal with his spare time,A,False
3,1032,Context: They may be small and not able to spe...,0. babies can learn even while asleep,A,False
4,1032,Context: They may be small and not able to spe...,0. babies can judge a person helpful or harmful,A,False


## Validator

### Baseline

In [ ]:
fn_sfx = "" # filename suffix, used when saving validator file
sys_prompt = "Answer 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect."

### Search for optimal prompt

In [6]:
fn_sfx = "_prmpt" # filename suffix, used when saving validator file
# sys_prompt = "Think step by step. First generate your own answer to the question. Next reflect on your answer. Finally, compare this with the provided answer. Check, then double check your thinking. The last word of your response should be 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect, given the question."
sys_prompt = "Think step by step. First generate your own answer to the question and then compare this with the provided answer. Check, then double check your thinking. The last word of your response should be 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect, given the question."

### Run validator

In [9]:
# gen_data_fn = f"results/{st_fn}.csv" # change this to the filename of the generator results
gen_data_fn = 'results/2024_02_13_race_n200_gpt-4-1106-preview.csv'

df = pd.read_csv(gen_data_fn)
indices = df.iloc[:, 0].tolist()
indices = indices[:-1] # just because we manually added an additional row on the CSV when calculating scores

val_preds = [] # list of predictions for validator, reset after each set of runs
val_evals = [] # whether prediction matches answer


# Evaluate
for _, row in df.iterrows():
    idx = row['Index']
    ans = row['Prediction']
    gold = row['Eval'] # change to Manual Eval in other settings
    question = row['Question']
    
    q = "Question: " + question + " "
    q += f"Answer: {ans}"
    logger.info(q)
    msg = [{"role": "system", "content": "The following are a series of multiple choice questions. Respond 'TRUE' if the answer is correct and 'FALSE' if incorrect. Respond only with one word."}, {"role": "user", "content": q}]
    val_preds.append(gpt.gen(msgs=msg).content)
    val_evals.append(check_bool(val_preds[-1], gold))
    print(val_preds[-1])

# Print the predictions and answers
logger.info(val_preds)
logger.info(df['Eval'].tolist())
percent_correct = (sum(val_evals) / len(val_evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# add validator predictions to column
df['Validator Pred'] = val_preds

# save the dataframe to csv
df.to_csv(f'results/{st_fn}_with_val{fn_sfx}.csv', index=False);

2024-02-14 18:05:59,694 - INFO - Question: Context: Some nations think they must have more and more babies, more and more people, if they are to remain strong and free.
Actually, this is not so. Very often in history, small nations have conquered large ones. It's not so much the size of the army as its organization and the technical level of its weapons. Thus, Greece took over Persia in the 300s B.C., Great Britain took over India in the 1700s, even though Persia and India had far bigger population than Greece and Great Britain.
If a nation wishes to avoid being dominated by its neighbors, its best chance is to raise its standard of living and its level of technology. _ can be done best by not allowing its population to grow to such a point that it is sunk in misery and poverty. In fact, the worst way in which a nation can try to avoid being dominated by its neighbor is to increase its population to the point of misery and poverty.
If every nation tries to compete with its neighbors by

TRUE


2024-02-14 18:06:00,998 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 18:06:01,000 - INFO - Question: Context: Some nations think they must have more and more babies, more and more people, if they are to remain strong and free.
Actually, this is not so. Very often in history, small nations have conquered large ones. It's not so much the size of the army as its organization and the technical level of its weapons. Thus, Greece took over Persia in the 300s B.C., Great Britain took over India in the 1700s, even though Persia and India had far bigger population than Greece and Great Britain.
If a nation wishes to avoid being dominated by its neighbors, its best chance is to raise its standard of living and its level of technology. _ can be done best by not allowing its population to grow to such a point that it is sunk in misery and poverty. In fact, the worst way in which a nation can try to avoid being dominated by its neighbor is to i

TRUE


2024-02-14 18:06:01,411 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 18:06:01,413 - INFO - Question: Context: Joe Temper should have known better. After all, he works for a large auto insurance company. It won't hurt to leave the key in the truck this once, he thought, as he filled his gas tank at a self-service gas station. But minutes later, as he was paying the money, he saw the truck being driven away.
In 2010, 1.6 million motor vehicles were stolen in the United States --one every 20 seconds. If present trends continue, experts predict vehicle thefts   could be over two million every year by the end of the decade.
Vehicle theft is a common phenomenon, which has a direct effect on over four million people a year. The cost is astonishing.
Many police officials blame professional thieves for the large number of the thefts. It is a major moneymaker for organized crime. Typically, stolen cars are taken to pieces and the parts sold 

...

TRUE


2024-02-14 18:10:16,589 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 18:10:16,591 - INFO - Question: Context: With thousands of waterfalls across the world, it is difficult to choose the best but the following list contains some of the world's most amazing and beautiful ones!
Iguazu Falls-The Iguazu Falls is located on the border of Brazil and Argentina and it is well-known for its scenic beauty. Though it is just about 89 meters high, it ranks as the fourth best waterfall in the world because of its beauty. The U-shaped cliff in the waterfall which is known as the Devil's throat is the most striking feature of the waterfall.
Angel Falls-The Angel Falls, named after its discoverer James Crawford Angel, is the world's highest waterfalls, falling from a height of 979 meters. It is located in the Bolivar state in Venezuela. A unique feature of the falls is that most of the water falling down gets blown away by the winds as mist . The 

TRUE


2024-02-14 18:10:16,967 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 18:10:16,969 - INFO - ['TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE'

TRUE
Percentage of True values: 90.21406727828746%
